In [14]:
# # from dash import Dash, html
# # import dash_ag_grid as dag
# # import plotly.express as px

# # app = Dash(__name__)

# # df = px.data.gapminder()
# # df = df[['country', 'continent', 'year', 'pop']]

# # app.layout = html.Div(
# #     [
# #         dag.AgGrid(
# #             id="getting-started-themes-example",
# #             columnDefs= [{"field": x, } for x in df.columns],
# #             rowData= df.to_dict('records'),
# #             className="ag-theme-balham",
# #             columnSize="sizeToFit",
# #         ),
# #     ]
# # )

# # if __name__ == "__main__":
# #     app.run(debug=True)

# from dash import Dash, html
# import dash_ag_grid as dag
# import pandas as pd

# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv")

# app = Dash(__name__)

# columnDefs = [
#     { 'field': 'country', 'filter': True },
#     { 'field': 'lifeExp', 'headerName': 'Life Expectancy', 'filter': True },
#     { 'field': 'pop', 'headerName': 'Population'},    
# ]

# grid = dag.AgGrid(
#     id="getting-started-filter",
#     rowData=df.to_dict("records"),
#     columnDefs=columnDefs,
#     className="ag-theme-balham",    
# )

# app.layout = html.Div([grid])

# if __name__ == "__main__":
#     app.run(debug=True)


In [2]:
import dash
import dash_ag_grid as dag
from dash import html, dcc, Input, Output, State, no_update
import pandas as pd
import plotly.express as px

# Sample data
data = {
    "Name": ["Series 1", "Series 2", "Series 3", "Series 4", "Series 5"],
    "2024-07-10": [10, 20, 30, 40, 50],
    "2024-07-11": [15, 25, 35, 45, 55],
    "2024-07-12": [20, 30, 40, 50, 60],
    "2024-07-13": [25, 35, 45, 55, 65],
    "2024-07-14": [30, 40, 50, 60, 70],
    "Selected": [""] * 5
}
df = pd.DataFrame(data)

app = dash.Dash(__name__)

app.layout = html.Div([
    dag.AgGrid(
        id='data-table',
        columnDefs=[
            {'headerName': '', 'field': 'Selected', 'cellRenderer': 'selectedCellRenderer', 'width': 50},
            {'headerName': 'Name', 'field': 'Name', 'sortable': True},
            {'headerName': '2024-07-10', 'field': '2024-07-10', 'sortable': True},
            {'headerName': '2024-07-11', 'field': '2024-07-11', 'sortable': True},
            {'headerName': '2024-07-12', 'field': '2024-07-12', 'sortable': True},
            {'headerName': '2024-07-13', 'field': '2024-07-13', 'sortable': True},
            {'headerName': '2024-07-14', 'field': '2024-07-14', 'sortable': True},
        ],
        rowData=df.to_dict('records'),
        className="ag-theme-balham",
        columnSize="sizeToFit",
        dangerously_allow_code=True,
    ),
    dcc.Graph(id='line-graph'),
    dcc.Store(id='hovered-column')
])

@app.callback(
    Output('data-table', 'rowData'),
    [Input('data-table', 'cellClicked')],
    [State('data-table', 'rowData')]
)
def update_check_mark(cellClicked, rows):
    if cellClicked is not None:
        row_index = cellClicked['rowIndex']
        for i in range(len(rows)):
            rows[i]['Selected'] = ''
        rows[row_index]['Selected'] = '✔️'
    return rows

@app.callback(
    Output('line-graph', 'figure'),
    [Input('data-table', 'cellClicked')]
)
def display_graph(cellClicked):
    if cellClicked is not None:
        row_index = cellClicked['rowIndex']
        selected_series = df.iloc[row_index]
        series_name = selected_series['Name']
        series_data = {
            "Date": selected_series.index[1:-1],
            "Value": selected_series.values[1:-1]
        }
        series_df = pd.DataFrame(series_data)
        fig = px.line(series_df, x='Date', y='Value', title=f'Line Graph for {series_name}')
        return fig
    return {}

@app.callback(
    Output('hovered-column', 'data'),
    [Input('data-table', 'cellMouseOver')]
)
def update_hovered_column(cellMouseOver):
    if cellMouseOver is not None:
        return cellMouseOver['column']['colId']
    return no_update

@app.callback(
    Output('data-table', 'columnDefs'),
    [Input('hovered-column', 'data')],
    [State('data-table', 'columnDefs')]
)
def update_column_styles(hovered_column, column_defs):
    if hovered_column:
        for col in column_defs:
            if col['field'] == hovered_column:
                col['cellStyle'] = {'backgroundColor': 'red'}
            else:
                col['cellStyle'] = {}
    return column_defs

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
